# Drazin Inverse

In [44]:
from numpy.linalg import matrix_power
import numpy as np
import scipy.linalg

### Problem 1

In [45]:
def is_drazin(A, k, A_D):
    return np.allclose(A @ A_D, A_D @ A) & \
           np.allclose(matrix_power(A, k+1) @ A_D, matrix_power(A, k)) & \
           np.allclose(A_D @ A @ A_D, A_D)

In [46]:
A = np.array([[1, 3, 0, 0],
              [0, 1, 3, 0],
              [0, 0, 1, 3],
              [0, 0, 0, 0]])
k = 1
A_D = np.array([[1, -3, 9, 81],
                [0, 1, -3, -18],
                [0, 0, 1, 3],
                [0, 0, 0, 0]])

is_drazin(A, k, A_D)

True

In [47]:
B = np.array([[1, 1, 3],
              [5, 2, 6],
              [-2, -1, -3]])

k = 3

B_D = np.zeros((3, 3))

is_drazin(B, k, B_D)

True

### Problem 2

In [48]:
def drazin(A, tol):
    
    n, n = A.shape
    
    g = lambda x: abs(x) > tol
    leq = lambda x: abs(x) <= tol
    Q1, S, k1 = scipy.linalg.schur(A, sort=g)
    Q2, T, k2 = scipy.linalg.schur(A, sort=leq)
    
    U = np.hstack([S[:, :k1], T[:, :(n-k1)]])
    U_inv = np.linalg.inv(U)
    V = U_inv @ A @ U
    Z = np.zeros((n, n))
    
    if k != 0:
        M_inv = np.linalg.inv(V[:k1, :k1])
        Z[:k1, :k1] = M_inv
    
    return U @ Z @ U_inv

In [49]:
A = np.random.random((3, 3))

In [50]:
AD = drazin(A, 1e-3)

In [51]:
np.abs(AD @ A).round()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [52]:
is_drazin(AD, 5, A)

True

### Problem 3

In [138]:
G1 = np.array([[0, 1, 0, 0], 
               [1, 0, 1, 0], 
               [0, 1, 0, 1], 
               [0, 0, 1, 0]])
G2 = np.array([[0, 1], 
               [1, 0]])

In [139]:
def resistance_mat(A):
    
    n, n = A.shape
    D = np.diag(A.sum(axis=1))
    L = D - A
    R = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            L_tilde = L.copy()
            L_tilde[j, :] = np.eye(n)[j]
            L_tilde = drazin(L_tilde, 1e-5)
            if i != j:
                R[i, j] = L_tilde[i, i]
    
    return R

In [140]:
resistance_mat(G1).round()

array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 0., 1.],
       [3., 2., 1., 0.]])

### Problem 4 & 5

In [427]:
class LinkPredictor:
    
    def __init__(self, filename):
        
        self.filename = filename
        
        nodes = []
        with open('social_network.csv', 'r') as f:
            for line in f:
                nodes.append(line.strip().split(','))
        nodes = np.array(nodes)
        names = np.unique(nodes)
        N = len(names)
        names_id = dict(zip(names, range(N)))
        nodes_id = np.array([[names_id[i], names_id[j]] for i, j in nodes[:, ]])
        
        A = np.zeros((N, N))

        for pair in nodes_id:
            i, j = pair
            A[i, j] = 1
            A[j, i] = 1
            
        self.A = A
        self.names = names
        self.R = resistance_mat(A)
        
    def predict_link(self, node=None):
        R, A = self.R, self.A
        names = self.names
        R[A == 1] = 0
        if node is None:
            min_resist = np.min(R[R > 0])
            loc = np.argwhere(R == min_resist).flatten()
            return (names[loc[0]], names[loc[1]])
        elif node.isalpha():
            if node not in names:
                raise ValueError('Node not in network')
            i = np.argwhere(names == node)
            R_i = R[:, i]
            min_resist = np.min(R_i[R_i > 0])
            loc = np.argwhere(R_i == min_resist).flatten()
            return (node, names[loc[0]])
        
    def add_link(self, node1, node2):
        names = self.names
        if node1 not in names or node2 not in names:
            raise ValueError('One of nodes is not in network')
        i = np.argwhere(names == node1)
        j = np.argwhere(names == node2)
        self.A[i, j] = 1
        self.A[j, i] = 1
        self.R = resistance_mat(self.A)

In [428]:
sn = LinkPredictor('social_network.csv')

In [429]:
sn.predict_link()

('Oliver', 'Emily')

In [430]:
sn.predict_link('Melanie')

('Melanie', 'Carol')

In [431]:
sn.predict_link('Alan')

('Alan', 'Sonia')

In [432]:
sn.add_link('Sonia', 'Alan')

In [433]:
sn.predict_link('Alan')

('Alan', 'Piers')

In [434]:
sn.add_link('Piers', 'Alan')

In [435]:
sn.predict_link('Alan')

('Alan', 'Abigail')